In [ ]:
import math 
import numpy as np 
import matplotlib.pyplot as plt
import random
%matplotlib inline 

In [ ]:
words = open('names.txt', 'r').read().splitlines()

In [ ]:
import torch


chars  = sorted(list(set([ch for w in words for ch in w])))
stoi = {ch:i+1 for i, ch in enumerate(chars)}
stoi['.'] = 0

itos = {i+1:ch for i, ch in enumerate(chars)}
itos[0] = '.'



In [ ]:
#bigram model 
b = {}

N = torch.ones((27,27), dtype=torch.int32) #We take ones as a smoothing factor

for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        N[ix1, ix2] += 1
        



In [ ]:
plt.figure(figsize=(16,16))
plt.imshow(N, cmap='Blues')
for i in range(27):
    for j in range(27):
        chstr = itos[i] + itos[j]
        plt.text(j, i, chstr, ha="center", va="bottom", color='gray')
        plt.text(j, i, N[i, j].item(), ha="center", va="top", color='gray')
plt.axis('off')

In [ ]:

P = N.float() 
P /= P.sum(dim=1, keepdim=True)

In [ ]:
g = torch.Generator().manual_seed(2147483647)

for i in range(20):
    ix = 0
    out = []
    while True:
        p = P[ix].float()
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        ch = itos[ix]
        if ch == '.':
            break
        out.append(ch)

    print(''.join(out))

In [ ]:
#We are going to write out a simple loss function by taking the averaage negative log likelihood of the next character in the sequence.
loss = 0
n = 0



for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        loss -= math.log(P[ix1, ix2].item())
        n += 1

print(loss/n)



In [114]:
#Create training set of all the bigrams (x,y) where x is the input and y is the output
xs, ys = [], []
for w in words[:1]:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
print(xs)
print(ys)

import torch.nn.functional as F
x_enc = F.one_hot(xs, num_classes=27).float()



tensor([ 0,  5, 13, 13,  1])
tensor([ 5, 13, 13,  1,  0])


In [130]:
W = torch.randn((27,27))
x_enc @ W

tensor([[-0.6702],
        [ 0.1416],
        [-1.9431],
        [-1.9431],
        [-0.4150]])